# LambdaCat Cookbook

Working examples of core LambdaCat features.
Each example can be run independently.

## Setup

Import the necessary modules:

In [ ]:
from LambdaCat.agents.actions import Task, parallel, sequence
from LambdaCat.agents.runtime import compile_plan, compile_to_kleisli
from LambdaCat.core import arrow, build_presentation, obj
from LambdaCat.core.category import Cat
from LambdaCat.core.diagram import Diagram
from LambdaCat.core.fp.instances.option import Option
from LambdaCat.core.fp.instances.reader import Reader
from LambdaCat.core.fp.instances.result import Result
from LambdaCat.core.fp.instances.state import State
from LambdaCat.core.fp.instances.writer import Writer
from LambdaCat.core.fp.kleisli import Kleisli
from LambdaCat.core.ops_category import check_commutativity
from LambdaCat.core.optics import Lens

print("LambdaCat modules imported")

## Snippet 1: Writer Logging

Log accumulation with monoid

In [ ]:
# Create a monoid for log accumulation
class StringMonoid:
    def empty(self):
        return ""
    def combine(self, a, b):
        return a + b

# Create monoid instance
string_monoid = StringMonoid()

# Logged computation
def increment_logged(x):
    return Writer(x + 1, f"Incremented {x} to {x + 1}\n", string_monoid)

def double_logged(x):
    return Writer(x * 2, f"Doubled {x} to {x * 2}\n", string_monoid)

# Compose with logging
result = increment_logged(5).bind(double_logged)
print(f"Value: {result.value}")  # 12
print(f"Log: {result.log}")      # "Incremented 5 to 6\nDoubled 6 to 12\n"

## Snippet 2: Reader Config

Configuration-dependent computation

In [ ]:
# Configuration-dependent computation
def get_user_name(user_id):
    return Reader(lambda config: config["users"].get(user_id, "Unknown"))

def get_user_age(user_id):
    return Reader(lambda config: config["ages"].get(user_id, 0))

# Compose readers
def user_info(user_id):
    return get_user_name(user_id).bind(
        lambda name: get_user_age(user_id).map(
            lambda age: f"{name} is {age} years old"
        )
    )

# Run with configuration
config = {"users": {1: "Alice", 2: "Bob"}, "ages": {1: 30, 2: 25}}
result = user_info(1).run(config)
print(result)  # "Alice is 30 years old"

## Snippet 3: Result Validation

Applicative vs Monad validation patterns

In [ ]:
# Applicative: parallel validation
def validate_age(age):
    if age < 0:
        return Result.err("Age cannot be negative")
    if age > 150:
        return Result.err("Age seems unrealistic")
    return Result.ok(age)

def validate_name(name):
    if not name or len(name.strip()) == 0:
        return Result.err("Name cannot be empty")
    return Result.ok(name.strip())

# Applicative parallel validation
age_result = validate_age(25)
name_result = validate_name("Alice")

# Both validations run independently
validation_func = Result.ok(lambda age: lambda name: {"age": age, "name": name})
result = validation_func.ap(age_result).ap(name_result)

print(f"Parallel validation: {result}")
# Output: Result.ok({'age': 25, 'name': 'Alice'})

# Monad: sequential validation (stops on first error)
def validate_user(user_data):
    return validate_age(user_data["age"]).bind(
        lambda age: validate_name(user_data["name"]).map(
            lambda name: {"age": age, "name": name}
        )
    )

# This will fail on first error
bad_result = validate_user({"age": -5, "name": "Alice"})
print(f"Sequential validation: {bad_result}")
# Output: Result.err('Age cannot be negative')

## Snippet 4: State Counters

Stateful operations with composition

In [ ]:
# Stateful operations
def increment_counter():
    return State(lambda s: (s + 1, s + 1))

def double_counter():
    return State(lambda s: (s * 2, s * 2))

def reset_counter():
    return State(lambda s: (0, 0))

# Compose stateful operations
def complex_counter():
    return increment_counter().bind(
        lambda _: increment_counter().bind(
            lambda _: double_counter().bind(
                lambda _: reset_counter()
            )
        )
    )

# Run with initial state
final_value, final_state = complex_counter().run(0)
print(f"Final value: {final_value}, Final state: {final_state}")
# Output: Final value: 0, Final state: 0
# State was: 0 → 1 → 2 → 4 → 0

## Snippet 5: List Search with Kleisli

Monadic list operations

In [ ]:
# Kleisli arrows for list operations
def find_item(predicate):
    return Kleisli(lambda xs:
        Option.some(next((x for x in xs if predicate(x)), None))
        .bind(lambda x: Option.some(x) if x is not None else Option.none())
    )

def filter_items(predicate):
    return Kleisli(lambda xs: Option.some([x for x in xs if predicate(x)]))

# Compose search operations
search_plan = find_item(lambda x: x > 10).compose(
    filter_items(lambda x: x % 2 == 0)
)

numbers = [5, 12, 8, 15, 20, 3]
result = search_plan(numbers)
print(f"Search result: {result}")
# Note: This may not work as expected due to composition order

## Snippet 6: Lenses on Nested Dict

Immutable state management

In [ ]:
# Create lenses for nested access
user_lens = Lens(
    get=lambda data: data.get("user", {}),
    set=lambda user, data: {**data, "user": user}
)

name_lens = Lens(
    get=lambda user: user.get("name", ""),
    set=lambda name, user: {**user, "name": name}
)

age_lens = Lens(
    get=lambda user: user.get("age", 0),
    set=lambda age, user: {**user, "age": age}
)

# Compose lenses
user_name_lens = user_lens.compose(name_lens)
user_age_lens = user_lens.compose(age_lens)

# Use lenses
data = {"user": {"name": "Alice", "age": 30}}
new_data = user_name_lens.set("Bob", data)
print(f"Updated name: {new_data}")
# Output: {'user': {'name': 'Bob', 'age': 30}}

# View through lens
age = user_age_lens.get(data)
print(f"Current age: {age}")  # 30

## Snippet 7: Commuting Triangle

Diagram construction and verification

In [ ]:
# Create a commuting triangle
A, B, C = obj("A"), obj("B"), obj("C")
f = arrow("f", "A", "B")
g = arrow("g", "B", "C")
h = arrow("h", "A", "C")

# Build category
C_triangle = build_presentation([A, B, C], [f, g, h])

# Create diagram
triangle_diagram = Diagram.from_edges(
    ["A", "B", "C"],
    [("A", "B", "f"), ("B", "C", "g"), ("A", "C", "h")]
)

# Check if triangle commutes (h = g ∘ f)
commutes = check_commutativity(Cat.from_presentation(C_triangle), "A", "C", [["f", "g"], ["h"]])
print(f"Triangle commutes: {commutes.ok}")  # True

# Render diagram
mermaid = triangle_diagram.to_mermaid()
print("Mermaid diagram:")
print(mermaid)

## Snippet 8: Failing Square

Non-commutative diagram example

In [ ]:
# Create a non-commuting square
A, B, C, D = obj("A"), obj("B"), obj("C"), obj("D")
f1 = arrow("f1", "A", "B")
f2 = arrow("f2", "C", "D")
g1 = arrow("g1", "A", "C")
g2 = arrow("g2", "B", "D")

# Build category
C_square = build_presentation([A, B, C, D], [f1, f2, g1, g2])

# Create diagram
square_diagram = Diagram.from_edges(
    ["A", "B", "C", "D"],
    [("A", "B", "f1"), ("C", "D", "f2"), ("A", "C", "g1"), ("B", "D", "g2")]
)

# Check commutativity (this will fail)
commutes = check_commutativity(Cat.from_presentation(C_square), "A", "D", [["f1", "g2"], ["g1", "f2"]])
print(f"Square commutes: {commutes.ok}")  # False

# Render failing diagram
dot = square_diagram.to_dot()
print("DOT diagram:")
print(dot)

## Snippet 9: Plan → Kleisli Agent

Agent compilation to monadic arrows

In [ ]:
# Define agent plan
increment_task = Task("increment")
double_task = Task("double")
plan = sequence(increment_task, double_task)

# Define actions
actions = {
    "increment": lambda x: x + 1,
    "double": lambda x: x * 2
}

# Compile to Kleisli arrow
kleisli_plan = compile_to_kleisli(actions, plan, Option)
result = kleisli_plan(5)

print(f"Agent result: {result}")
# Output: Option.some(12)

# Verify it's a proper monadic value
assert isinstance(result, Option)
assert result.is_some()
print("Agent compilation successful!")

## Snippet 10: Applicative Parallel Plan

Parallel execution with aggregation

In [ ]:
# Parallel plan
increment_task = Task("increment")
double_task = Task("double")
parallel_plan = parallel(increment_task, double_task)

# Compile with aggregation
def aggregate_results(results):
    return sum(results)  # Simple sum aggregation

actions = {
    "increment": lambda x: x + 1,
    "double": lambda x: x * 2
}

executable = compile_plan(actions, parallel_plan, aggregate_fn=aggregate_results)
result = executable(5)

print(f"Parallel result: {result}")
# Output: 16 (increment: 6, double: 10, sum: 16)

## Summary

Each snippet demonstrates a different LambdaCat feature:
1. **Writer logging** with monoid accumulation
2. **Reader configuration** dependency
3. **Result validation** (Applicative vs Monad)
4. **State counters** with composition
5. **List search** using Kleisli arrows
6. **Lenses** for nested data access
7. **Commuting triangle** diagrams
8. **Non-commutative** diagrams
9. **Agent plans** compiled to Kleisli
10. **Parallel execution** with aggregation

See the complete documentation for more examples.